In [43]:
import pandas as pd

import numpy as np

# import sqlalchemy

In [44]:
%cd C:\ALL_DATA\URGummitha\Desktop\Post_Mod-2\Run_M2.53

C:\ALL_DATA\URGummitha\Desktop\Post_Mod-2\Run_M2.53


# ============= USER INPUT  =============

### LOADING GRID POINT DATA FOR DFEM & GFEM [Created by Hypermesh]

In [45]:
dfem_nodes_csv_file = '../Run_M2.51/Fuse_RBE3Nodes.csv'

gfem_nodes_csv_file = '../Run_M2.51/Fuse_RBE3Nodes.csv'

# DFEM HDF5 [Input file name]

In [46]:
h5_file = './sol101_limit/pc12-47-v1-6-static_post-mod_m2.53_limit.h5'

# GFEM HDF5 [Input file name]

In [47]:
GFEM_h5_file = '../Run_M2.51/pc12-47-v1-6-static_post-mod_m3.1_limit.h5'

# =============== END OF USER INPUT ===============

# Loading HDF5 FILES

In [48]:
store = pd.HDFStore(h5_file, mode='r')

In [49]:
gfem_store = pd.HDFStore(GFEM_h5_file, mode='r')

In [31]:
dfem_wing_rbe3_nodes = pd.read_csv(dfem_nodes_csv_file)

gfem_wing_rbe3_nodes = pd.read_csv(gfem_nodes_csv_file)

In [9]:
def get_df(x, eid_array, cols = list('AB')):
    
    df = pd.DataFrame(x[:,[0,10]],index=eid_array, columns=cols)
    
    return df

In [10]:
def Join_DF(df1,df2):
    
    if df1.empty == True:
        
        df1 = df2.copy()
        
    else :
        
        df1 = pd.concat([df1, df2], axis=1, sort=False, join_axes=[df1.index])
        
    return df1

In [11]:
def CBAR_ELEMENTAL_FORCES(store):
    
    df = store.get('/NASTRAN/RESULT/ELEMENTAL/ELEMENT_FORCE/BAR')
    
    return df

In [12]:
def CBEAM_ELEMENTAL_FORCES(store):
    
    beam_node = store.get_node('/NASTRAN/RESULT/ELEMENTAL/ELEMENT_FORCE/BEAM')

    beam_lst_names = beam_node.colnames[1:-1]

    res_df = pd.DataFrame({'A' : []})

    for x in beam_lst_names:

        col_name_a = x + '_A'

        col_name_b = x + '_B'

        df = get_df(beam_node.col(x), beam_node.col('EID'), cols=[col_name_a, col_name_b])

        res_df = Join_DF(res_df, df)
        
    domain_id_df = pd.DataFrame(beam_node.col('DOMAIN_ID'), index=beam_node.col('EID'), columns=['DOMAIN_ID'])
    
    res_df = Join_DF(res_df, domain_id_df)
    
    res_df.index.name = 'EID'
    
    return res_df

In [13]:
def CELAS2_ELEMENTAL_FORCES(store):
    
    df = store.get('/NASTRAN/RESULT/ELEMENTAL/ELEMENT_FORCE/ELAS2')
    
    return df

In [14]:
def CQUAD4_ELEMENTAL_FORCES(store):
    
    df = store.get('/NASTRAN/RESULT/ELEMENTAL/ELEMENT_FORCE/QUAD4')
    
    return df

In [15]:
def CROD_ELEMENTAL_FORCES(store):
    
    df = store.get('/NASTRAN/RESULT/ELEMENTAL/ELEMENT_FORCE/ROD')
    
    return df

In [16]:
def CBUSH_ELEMENTAL_FORCES(store):
    
    df = store.get('/NASTRAN/RESULT/ELEMENTAL/ELEMENT_FORCE/BUSH')
    
    return df


In [17]:
def CSHEAR_ELEMENTAL_FORCES(store):
    
    df = store.get('/NASTRAN/RESULT/ELEMENTAL/ELEMENT_FORCE/SHEAR')
    
    return df

In [18]:
def CTRIA3_ELEMENTAL_FORCES(store):
    
    df = store.get('/NASTRAN/RESULT/ELEMENTAL/ELEMENT_FORCE/TRIA3')
    
    return df

In [19]:
def CBAR_ELEMENTAL_STRESS(store):
    
    df = store.get('/NASTRAN/RESULT/ELEMENTAL/STRESS/BAR')
    
    return df

In [20]:
def CBEAM_ELEMENTAL_STRESS(store):
    
    beam_node = store.get_node('/NASTRAN/RESULT/ELEMENTAL/STRESS/BEAM')

    beam_lst_names = beam_node.colnames[1:-1]

    res_df = pd.DataFrame({'A' : []})

    for x in beam_lst_names:

        col_name_a = x + '_A'

        col_name_b = x + '_B'

        df = get_df(beam_node.col(x), beam_node.col('EID'), cols=[col_name_a, col_name_b])

        res_df = Join_DF(res_df, df)
        
    domain_id_df = pd.DataFrame(beam_node.col('DOMAIN_ID'), index=beam_node.col('EID'), columns=['DOMAIN_ID'])
    
    res_df.index.name = 'EID'
    
    res_df = Join_DF(res_df, domain_id_df)
    
    return res_df

In [21]:
def CELAS2_ELEMENTAL_STRESS(store):
    
    df = store.get('/NASTRAN/RESULT/ELEMENTAL/STRESS/ELAS2')
    
    return df

In [22]:
def CQUAD4_ELEMENTAL_STRESS(store):
    
    df = store.get('/NASTRAN/RESULT/ELEMENTAL/STRESS/QUAD4')
    
    return df

In [23]:
def CROD_ELEMENTAL_STRESS(store):
    
    df = store.get('/NASTRAN/RESULT/ELEMENTAL/STRESS/ROD')
    
    return df

In [24]:
def CSHEAR_ELEMENTAL_STRESS(store):
    
    df = store.get('/NASTRAN/RESULT/ELEMENTAL/STRESS/SHEAR')
    
    return df

In [25]:
def CTRIA3_ELEMENTAL_STRESS(store):
    
    df = store.get('/NASTRAN/RESULT/ELEMENTAL/STRESS/TRIA3')
    
    return df

# DOMAIN_ID Mapping to SUBCASE

In [26]:
domain_results  =  store.get('/NASTRAN/RESULT/DOMAINS')

domain_results = domain_results.loc[:, ['ID', 'SUBCASE']].copy()

domain_results.set_index('ID', inplace=True)

subcase_mapping = domain_results.to_dict()

# subcase_mapping['SUBCASE']

# GRID POINT EXTRACTION

In [27]:
def GP_NODAL_DISPLACEMENT(store):
    
    df = store.get('/NASTRAN/RESULT/NODAL/DISPLACEMENT')
    
    return df

In [28]:
all_gp_dispacements = GP_NODAL_DISPLACEMENT(store)

# gp_dispacements.head()

### custom function to extract grid point forces for selected nodes.

In [29]:
def Custom_GP_Forces(all_gp_df, wing_gp_nodes_df ):

    filter_bool = all_gp_df.ID.isin(wing_gp_nodes_df.dependentnode.tolist())

    gp_dispacements = all_gp_df[filter_bool == True].copy()

    # gp_dispacements.head()

    gp_dispacements['SUBCASE'] = gp_dispacements.DOMAIN_ID.map(subcase_mapping['SUBCASE']).copy()

    # gp_dispacements.head()

    take_slice = wing_gp_nodes_df[['dependentnode', 'globalx', 'globaly', 'globalz']].copy()

    take_slice.columns = ['ID', 'X_Cord', 'Y_Cord', 'Z_Cord']

    # take_slice.head()

    merged_df  = gp_dispacements.merge(take_slice, on=['ID'], how='outer')

    # merged_df.head()

    merged_df.columns = ['DEPENDENT_NODE', 'X', 'Y', 'Z', 'RX', 'RY', 'RZ', 'DOMAIN_ID', 'SUBCASE', 'X_Cord',
           'Y_Cord', 'Z_Cord']

    # merged_df.head()

    merged_df['Rx_in_Degree'] = merged_df.RX * (180.0/np.pi)

    merged_df['Ry_in_Degree'] = merged_df.RY * (180.0/np.pi)

    merged_df['Rz_in_Degree'] = merged_df.RZ * (180.0/np.pi)
    
    # merged_df.to_csv('DFEM_wing_displacements.csv')
    
    return merged_df

In [32]:
def Custom_Switch_X_to_Y(df):
    
    new_column_order = ['DEPENDENT_NODE', 'X', 'Y', 'Z', 'RX', 'RY', 'RZ', 'DOMAIN_ID',
       'SUBCASE', 'Y_Cord', 'X_Cord', 'Z_Cord', 'Rx_in_Degree', 'Ry_in_Degree',
       'Rz_in_Degree']
    
    df = df.reindex(columns=new_column_order).copy()
    
    return df

In [33]:
# dfem_wing_rbe3_nodes = dfem_wing_rbe3_nodes[(dfem_wing_rbe3_nodes[['globalx', 'globaly', 'globalz']] != 0).all(axis=1)].copy()

In [34]:
# dfem_wing_rbe3_nodes.head()

In [35]:
dfem_extraction_df = Custom_GP_Forces(all_gp_dispacements, dfem_wing_rbe3_nodes)

dfem_extraction_df = Custom_Switch_X_to_Y(dfem_extraction_df)

# dfem_extraction_df.head()

In [120]:
# df = pd.DataFrame({'a':np.random.randn(3), 'b': np.random.randn(3), 'c':np.random.randn(3)})

# df.head()

In [121]:
# df = df.reindex(columns=['c','a','b'])

# df.head()

In [36]:
all_gfem_gp_displacemnts = GP_NODAL_DISPLACEMENT(gfem_store)

In [37]:
gfem_extraction_df = Custom_GP_Forces(all_gfem_gp_displacemnts, gfem_wing_rbe3_nodes)

gfem_extraction_df = Custom_Switch_X_to_Y(gfem_extraction_df)

# gfem_extraction_df.head()

In [38]:
with pd.ExcelWriter('FUSE_NODE_DISPLACEMENT_EXTRACTION.xlsx') as writer:
    
    dfem_extraction_df.to_excel(writer, sheet_name='DFEM_EXTRACTION')
    
    gfem_extraction_df.to_excel(writer, sheet_name='GFEM_EXTRACTION')

In [39]:
merged_df = dfem_extraction_df.merge(gfem_extraction_df, on=['DEPENDENT_NODE'], suffixes=('_DFEM', '_GFEM'))

# merged_df.head()

with pd.ExcelWriter('FUSE_NODE_DISPLACEMENT_EXTRACTION_COMBINED.xlsx') as writer:
    
    merged_df.to_excel(writer, sheet_name='COMBINED_DGFEM_EXTRACTION')

# Close the Entire Store HDF5

In [52]:
# closing the dfem store
store.close()

# closing the gfem store
gfem_store.close()

In [41]:
print('END OF PYTHON RUN.')

END OF PYTHON RUN.


In [85]:
%cd C:\ALL_DATA\URGummitha\Desktop\Post_Mod-2\Run_M2.54\sol101_ult

store = pd.HDFStore('./pc12-47-v1-6-static_post-mod_m2.54_ult.h5', mode='r')

store.keys()

solid_stress_node = store.get_node('/NASTRAN/RESULT/ELEMENTAL/STRESS/HEXA')

x_data = solid_stress_node.col('X')

x_data[1,:]

penta_stress_node  = store.get_node('/NASTRAN/RESULT/ELEMENTAL/STRESS/PENTA')

penta_stress_node.cols

x_data_penta = penta_stress_node.col('X')

x_data_penta[:2,:]

# store.close()

C:\ALL_DATA\URGummitha\Desktop\Post_Mod-2\Run_M2.54\sol101_ult


array([[  2.95823608,   6.06008601,  -3.59103584,   6.40567774,
          6.06572712,  -3.58671954,   6.39568097],
       [-11.09674679, -12.55657598, -10.26080829, -10.47302863,
        -12.55456655, -10.27519106, -10.46031023]])

In [86]:
store.keys()

['/NASTRAN/INPUT/DOMAINS',
 '/NASTRAN/RESULT/DOMAINS',
 '/NASTRAN/RESULT/NODAL/APPLIED_LOAD',
 '/NASTRAN/RESULT/NODAL/DISPLACEMENT',
 '/NASTRAN/RESULT/NODAL/GRID_FORCE',
 '/NASTRAN/RESULT/NODAL/GRID_WEIGHT',
 '/NASTRAN/RESULT/NODAL/MPC_FORCE',
 '/NASTRAN/RESULT/NODAL/SPC_FORCE',
 '/NASTRAN/RESULT/ELEMENTAL/ELEMENT_FORCE/BAR',
 '/NASTRAN/RESULT/ELEMENTAL/ELEMENT_FORCE/BEAM',
 '/NASTRAN/RESULT/ELEMENTAL/ELEMENT_FORCE/BUSH',
 '/NASTRAN/RESULT/ELEMENTAL/ELEMENT_FORCE/ELAS2',
 '/NASTRAN/RESULT/ELEMENTAL/ELEMENT_FORCE/QUAD4',
 '/NASTRAN/RESULT/ELEMENTAL/ELEMENT_FORCE/ROD',
 '/NASTRAN/RESULT/ELEMENTAL/ELEMENT_FORCE/SHEAR',
 '/NASTRAN/RESULT/ELEMENTAL/ELEMENT_FORCE/TRIA3',
 '/NASTRAN/RESULT/ELEMENTAL/STRESS/BAR',
 '/NASTRAN/RESULT/ELEMENTAL/STRESS/BEAM',
 '/NASTRAN/RESULT/ELEMENTAL/STRESS/BUSH',
 '/NASTRAN/RESULT/ELEMENTAL/STRESS/ELAS2',
 '/NASTRAN/RESULT/ELEMENTAL/STRESS/HEXA',
 '/NASTRAN/RESULT/ELEMENTAL/STRESS/PENTA',
 '/NASTRAN/RESULT/ELEMENTAL/STRESS/QUAD4',
 '/NASTRAN/RESULT/ELEMENTAL/ST

In [88]:
node_data_node = store.get_node('/NASTRAN/INPUT/NODE/GRID')

In [89]:
node_data_node.cols

/NASTRAN/INPUT/NODE/GRID.cols (Cols), 7 columns
  ID (Column(1533265,), int64)
  CP (Column(1533265,), int64)
  X (Column(1533265, 3), ('<f8', (3,)))
  CD (Column(1533265,), int64)
  PS (Column(1533265,), int64)
  SEID (Column(1533265,), int64)
  DOMAIN_ID (Column(1533265,), int64)

In [90]:
id_data, cp_data, cd_data = node_data_node.col('ID'), node_data_node.col('CP'),node_data_node.col('CD')

In [94]:
df = pd.DataFrame([pd.Series(id_data), pd.Series(cp_data), pd.Series(cd_data)])

In [ ]:
id_ser = pd.Series(id_data)

In [111]:
df = pd.DataFrame(id_ser, columns=['ID'])

In [112]:
df.head()

,ID
0,1
1,3
2,5
3,7
4,9
